In [7]:
import pandas as pd
import numpy as np

from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('../../data/tlp/preprocesado.csv')

In [9]:
df.head()

,demo-genero,demo-fechaEvaluacion,demo-fechaNacimiento,demo-rangoEdad,demo-edad,clin-reservaCognitiva_total,clin-reservaCognitiva_idiomas,clin-reservaCognitiva_ocupacion,clin-reservaCognitiva_escolaridad,clin-ansiedad,...,eval-TLP-Tavec-25_Discriminabilidad-PD,eval-TLP-Tavec-26_Sesgo-PD,eval-TLP-Tavec-27_RI_B_frente_RI_A1-PD,eval-TLP-Tavec-28_RL_CP_frente_RI_A5-PD,eval-TLP-Tavec-29_RCl_Cp_frente_RCl_LP-PD,eval-TLP-Tavec-30_RL_LP_frente_RL_CP-PD,eval-TLP-Tavec-31_RCl_LP_frente_RL_LP-PD,eval-TLP-Tavec-32_Recon_Ac_frente_RL_LP-PD,eval-TLP-Tavec-33_Recon_Ac_frente_RCl_LP-PD,ED_2Clases
0,2,2024-02-22T00:00:00,1942-05-08T00:00:00,Rango3,81,12,0,1,3,False,...,84.09,0.14,-20.00,0.00,33.33,37.50,-33.33,-38.46,-7.69,D
1,2,2024-06-26T00:00:00,1948-10-07T00:00:00,Rango3,75,8,0,1,3,True,...,79.55,-0.33,-71.43,12.50,0.00,0.00,0.00,-20.00,-20.00,D
2,2,2024-05-14T00:00:00,1952-05-08T00:00:00,Rango2,72,11,0,0,4,False,...,88.64,-0.60,-14.29,75.00,25.00,-20.00,0.00,-16.67,-16.67,H
3,2,2024-04-10T00:00:00,1964-01-27T00:00:00,Rango1,60,18,2,1,4,False,...,90.91,0.60,60.00,16.67,0.00,50.00,-20.00,-50.00,-37.50,D
4,2,2024-05-14T00:00:00,1959-09-16T00:00:00,Rango1,64,13,0,1,4,False,...,86.36,-0.71,0.00,66.67,0.00,-33.33,0.00,-10.00,-10.00,D


In [10]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train.shape, df_test.shape

((251, 80), (63, 80))

In [11]:
predictor= TabularPredictor(
    label='ED_2Clases',
    problem_type='binary',
    eval_metric='roc_auc',
    sample_weight='balance_weight',
    path='AutogluonModels/real/v2'
    )

In [12]:
predictor.fit(
    train_data = df_train,
    presets = ['high_quality'],
    time_limit = 900,
    auto_stack = True,
    excluded_model_types=['KNN','RF','XT', 'LR'],
    verbosity = 2
)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.2
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          12
Memory Avail:       5.85 GB / 15.94 GB (36.7%)
Disk Space Avail:   82.58 GB / 446.36 GB (18.5%)
Presets specified: ['high_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~8x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and 

In [13]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                               model  score_val eval_metric  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0                WeightedEnsemble_L3   0.957194     roc_auc       0.473018  274.051366                0.001085           0.043007            3      False         26
1               CatBoost_r177_BAG_L2   0.948919     roc_auc       0.311425  261.968276                0.044944         133.052523            2      False         23
2                WeightedEnsemble_L2   0.948835     roc_auc       0.209402  125.377173                0.000000           0.048656            2      False         16
3                    CatBoost_BAG_L2   0.944107     roc_auc       0.313946  163.226690                0.047465          34.310937            2      False         19
4                  LightGBMXT_BAG_L1   0.942165     roc_auc       0.052020    1.699996                0.052020   

{'model_types': {'LightGBMXT_BAG_L1': 'StackerEnsembleModel_LGB',
  'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'LightGBMLarge_BAG_L1': 'StackerEnsembleModel_LGB',
  'CatBoost_r177_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r131_BAG_L1': 'StackerEnsembleModel_LGB',
  'NeuralNetFastAI_r191_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r9_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'LightGBM_r96_BAG_L1': 'StackerEnsembleModel_LGB',
  'XGBoost_r33_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'CatBoost_r137_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'NeuralNetFastAI_r102_BAG_L1': 'StackerEnsembleModel_NNFastAiTabular',
  'CatBoost_r13_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBMXT_BAG_L2': 'StackerEnsembleModel_LGB',
  'LightGBM_BA

In [14]:
auxiliary_metrics = ['balanced_accuracy', 'f1', 'f1_macro', 'f1_micro', 'roc_auc', 'average_precision', 'precision', 'recall', 'log_loss', 'pac_score']

In [15]:
predictor.evaluate(
    df_test,
    silent = False,
    auxiliary_metrics = auxiliary_metrics
    )

Evaluation: roc_auc on test data: 0.9183006535947712
Evaluations on test data:
{
    "roc_auc": 0.9183006535947712,
    "accuracy": 0.8571428571428571,
    "balanced_accuracy": 0.7843137254901961,
    "mcc": 0.5517557332192131,
    "f1": 0.64,
    "precision": 0.6153846153846154,
    "recall": 0.6666666666666666
}


{'roc_auc': 0.9183006535947712,
 'accuracy': 0.8571428571428571,
 'balanced_accuracy': 0.7843137254901961,
 'mcc': 0.5517557332192131,
 'f1': 0.64,
 'precision': 0.6153846153846154,
 'recall': 0.6666666666666666}

In [16]:
predictors = predictor.leaderboard(
    df_train,
    extra_metrics = auxiliary_metrics,
    extra_info=True,
    silent=True
)

In [17]:
predictors

,model,score_test,balanced_accuracy,f1,f1_macro,f1_micro,roc_auc,average_precision,precision,recall,...,hyperparameters,hyperparameters_fit,ag_args_fit,features,compile_time,child_hyperparameters,child_hyperparameters_fit,child_ag_args_fit,ancestors,descendants
0,CatBoost_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.15, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[eval-TLP-Tavec-15_ESem_RL_LP-PD, eval-TLP-Tavec-1_RI_A1-PD, clin-tipoAlcohol.lower_ratio, ques-MMSE-fijacion-PD, eval-TLP-Tavec-8_RL_CP-PD, demo-fechaNacimiento.day, clin-tipoAlcohol.special_ratio, eval-TLP-Tavec-19_ESer_RL_LP-PD, ques-NavEspacial-resultadosWFQ_SA-PD, eval-TLP-Tavec-26_Sesgo-PD, eval-TLP-Tavec-4_RI_B-PD, clin-tipoAlcohol.word_count, clin-tipoAlcohol.symbol_count. , clin-tipoAlcohol.digit_ratio, ques-MMSE-Total-PD, eval-TLP-FigRey-Duracion-PD, eval-TLP-Tavec-24_FP-PD, ques-MMSE-escolaridad-PD, clin-alcohol, demo-fechaEvaluacion.month, clin-tipoAlcohol.symbol_ratio. , eval-...",None,"{'iterations': 214, 'learning_rate': 0.05, 'random_seed': 0, 'allow_writing_files': False, 'eval_metric': 'Logloss'}",{'iterations': 214},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}",[],[]
1,LightGBM_BAG_L1_FULL,1.000000,0.984043,0.954545,0.969165,0.976096,1.000000,1.000000,0.913043,1.000000,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.15, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': None, 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None, 'drop_unique': False}","[eval-TLP-Tavec-15_ESem_RL_LP-PD, eval-TLP-Tavec-1_RI_A1-PD, clin-tipoAlcohol.lower_ratio, ques-MMSE-fijacion-PD, eval-TLP-Tavec-8_RL_CP-PD, demo-fechaNacimiento.day, clin-tipoAlcohol.special_ratio, eval-TLP-Tavec-19_ESer_RL_LP-PD, ques-NavEspacial-resultadosWFQ_SA-PD, eval-TLP-Tavec-26_Sesgo-PD, eval-TLP-Tavec-4_RI_B-PD, clin-tipoAlcohol.word_count, clin-tipoAlcohol.symbol_count. , clin-tipoAlcohol.digit_ratio, ques-MMSE-Total-PD, eval-TLP-FigRey-Duracion-PD, eval-TLP-Tavec-24_FP-PD, ques-MMSE-escolaridad-PD, clin-alcohol, demo-fechaEvaluacion.month, clin-tipoAlcohol.symbol_ratio. , eval-...",None,"{'learning_rate': 0.05, 'num_boost_round': 72}",{'num_boost_round': 72},"{'max_memory_usage_ratio': 1.0, 'max_time_limit_ratio': 1.0, 'max_time_limit': None, 'min_time_limit': 0, 'valid_raw_types': ['bool', 'int', 'float', 'category'], 'valid_special_types': None, 'ignored_type_group_special': None, 'ignored_type_group_raw': None, 'get_features_kwargs': None, 'get_features_kwargs_extra': None, 'predict_1_batch_size': None, 'temperature_scalar': None}",[],[]
2,LightGBMXT_BAG_L1_FULL,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,"{'use_orig_features': True, 'valid_stacker': True, 'max_base_models': 0, 'max_base_models_per_type': 'auto', 'save_bag_folds': True}",{},"{'max_memory_usage_ratio': 1.15, 'max_tim

In [18]:
predictor.feature_importance(data=df_train)

These features in provided data are not utilized by the predictor and will be ignored: ['clin-Covid_sintomaSarpullido']
Computing feature importance via permutation shuffling for 78 features using 251 rows with 5 shuffle sets...
	76.48s	= Expected runtime (15.3s per shuffle set)
	22.63s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
eval-TLP-Stroop-color-PD,0.055167,0.008467,0.000065,5,0.072600,0.037734
demo-fechaNacimiento,0.000946,0.000647,0.015412,5,0.002278,-0.000386
eval-TLP-CubCorsi-totalDirectos-PD,0.000321,0.000210,0.013483,5,0.000754,-0.000112
demo-genero,0.000000,0.000000,0.500000,5,0.000000,0.000000
eval-TLP-Tavec-6_Rg_Md-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
...,...,...,...,...,...,...
ques-Sus-totalPar-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
ques-NavEspacial-resultadosWFQ_DE-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
ques-NavEspacial-resultadosWFQ_SA-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
ques-NavEspacial-resultadosWFQ_NO-PD,0.000000,0.000000,0.500000,5,0.000000,0.000000
